In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

import re
import math
import gspread
import pandas as pd
from google.colab import auth
from selenium import webdriver
from IPython.display import clear_output 
from oauth2client.client import GoogleCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
driver =webdriver.Chrome('chromedriver',chrome_options=chrome_options)

clear_output()

In [ ]:
def get_keyword():
  auth.authenticate_user()
  gc = gspread.authorize(GoogleCredentials.get_application_default())
  wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/10qKxpIl2IADvZqOoxrJRofz6M8_wikBL8JhsuGWfsKE/edit#gid=0')
  worksheet = wb.worksheet("Sheet1")
  key_word_list = worksheet.col_values(1)
  return key_word_list

def listToString(s): 
    str1 = "-" 
    return (str1.join(s))

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

main_url = 'https://vinmart.com/searchpves/searchkeywork'
kw_list = []
name_list = []
price_list = []
unit_list = []
unit = ['hộp','chai','gói','lốc','thùng','bộ','lon','bịch','hũ']
times = 100
for key_word in get_keyword():
  if key_word != 'keyword':
    key_word = key_word.replace(" ","-")
    url = main_url.replace("searchkeywork", key_word)
    for i in range(times):
      final_url = url + "?p=" + str(i+1)
      # print(final_url)
      driver.get(final_url)
      # print(len(driver.find_elements_by_class_name("product-name")))
      if len(driver.find_elements_by_class_name("product-name"))== 0:
        # print("no result")
        break
      else:
        all_name = driver.find_elements_by_class_name("product-name")
        all_price = driver.find_elements_by_class_name("product-price")
        if len(all_name) == len(all_price):
          for name in all_name:
            unit_tmp = []
            name_list.append(name.text)
            kw_list.append(key_word)
            for i in unit:
              if str(name.text).lower().find(i)>= 0:
                unit_tmp.append(i)
            unit_list.append(listToString(unit_tmp))          
          for price in all_price:
            price_list.append(price.text)
        else:
          print("len no match name = "+str(len(all_name))+" price = "+str(len(all_price)))
    if len(price_list) != len(name_list):
      break
      # finally:
      #     print("time out")


In [ ]:
df = pd.DataFrame({'keyword':kw_list, 'name': name_list, 'price': price_list, 'unit': unit_list})

auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/10qKxpIl2IADvZqOoxrJRofz6M8_wikBL8JhsuGWfsKE/edit#gid=0')
worksheet = wb.worksheet("raw")
set_with_dataframe(worksheet, df)